In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T

print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'cat': 3, 'dog': 5, 'bird': 2, 'ship': 8, 'truck': 9, 'deer': 4, 'automobile': 1, 'horse': 7, 'airplane': 0, 'frog': 6}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
print(X_validation.shape)
print(X_test.shape)
arr = np.arange(X_train.shape[0])
np.random.shuffle(arr)
print(arr)

(5000, 3072)
(10000, 3072)
[28249 42121  3225 ..., 33047 44680 24607]


In [6]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[0]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [7]:
logdir = 'tf_logs/summary9'
iteration = 20000
batch_size = 128
learning_rate = 0.01

In [8]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [9]:
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1')
        batch_norm1 = tf.layers.batch_normalization(conv1, axis = 1, name='batch_norm1')
        pool1 = tf.layers.max_pooling2d(inputs=batch_norm1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv2')
        batch_norm2 = tf.layers.batch_normalization(conv2, axis = 1, name='batch_norm2')
        pool2 = tf.layers.max_pooling2d(inputs=batch_norm2, pool_size=2, strides=2, padding='SAME', name='pool2')
        
    with tf.name_scope('conv3') as scope:
        conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv3')
        batch_norm3 = tf.layers.batch_normalization(conv3, axis = 1, name='batch_norm3')
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2, padding='SAME', name='pool3')

    with tf.name_scope('fc') as scope:
        pool3_flat = tf.reshape(pool3, [-1, 4*4*128], name='pool3_flat')
#         dropout1 = tf.layers.dropout(pool3_flat, rate=dropout_rate, training=training, name='dropout1')
        dense1 = tf.layers.dense(pool3_flat, units = 8*128, activation = tf.nn.relu, name='dense1')
        batch_norm4 = tf.layers.batch_normalization(dense1, axis = 1, name='batch_norm4')
#         dropout2 = tf.layers.dropout(dense1, rate=dropout_rate, training=training, name='dropout2')
        dense2 = tf.layers.dense(batch_norm4, units = 128, activation = tf.nn.relu, name='dense2')
#         dropout3 = tf.layers.dropout(dense2, rate=dropout_rate, training=training, name='dropout3')
        batch_norm5 = tf.layers.batch_normalization(dense2, axis = 1, name='batch_norm5')

    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(batch_norm5, units = 10, name='logits')

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    with tf.name_scope('summary') as scope:
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [10]:
def train(X_train, y_train, X_validation, y_validation, iteration):

    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
            

In [11]:
train(X_train, y_train, X_validation, y_validation, iteration)

###################################
validation! after 0 iterations the loss is 2.3025, the accuracy is 0.095
training! after 0 iterations the loss is 2.29372, the accuracy is 0.101562
###################################
validation! after 100 iterations the loss is 2.28119, the accuracy is 0.1958
training! after 100 iterations the loss is 2.28198, the accuracy is 0.164062
###################################
validation! after 200 iterations the loss is 2.24859, the accuracy is 0.1822
training! after 200 iterations the loss is 2.25003, the accuracy is 0.179688
###################################
validation! after 300 iterations the loss is 2.16826, the accuracy is 0.2376
training! after 300 iterations the loss is 2.14495, the accuracy is 0.25
###################################
validation! after 400 iterations the loss is 2.07501, the accuracy is 0.2382
training! after 400 iterations the loss is 2.024, the accuracy is 0.273438
###################################
validation! after 500 iter

###################################
validation! after 4400 iterations the loss is 1.22042, the accuracy is 0.5682
training! after 4400 iterations the loss is 1.03163, the accuracy is 0.640625
###################################
validation! after 4500 iterations the loss is 1.2995, the accuracy is 0.532
training! after 4500 iterations the loss is 1.0314, the accuracy is 0.6875
###################################
validation! after 4600 iterations the loss is 1.25211, the accuracy is 0.5508
training! after 4600 iterations the loss is 0.925062, the accuracy is 0.632812
###################################
validation! after 4700 iterations the loss is 1.2259, the accuracy is 0.5606
training! after 4700 iterations the loss is 1.13242, the accuracy is 0.59375
###################################
validation! after 4800 iterations the loss is 1.30632, the accuracy is 0.5276
training! after 4800 iterations the loss is 1.03795, the accuracy is 0.648438
###################################
validation

###################################
validation! after 8700 iterations the loss is 1.05199, the accuracy is 0.6436
training! after 8700 iterations the loss is 0.629409, the accuracy is 0.765625
###################################
validation! after 8800 iterations the loss is 1.04583, the accuracy is 0.6412
training! after 8800 iterations the loss is 0.619366, the accuracy is 0.804688
###################################
validation! after 8900 iterations the loss is 0.994326, the accuracy is 0.6636
training! after 8900 iterations the loss is 0.584644, the accuracy is 0.820312
###################################
validation! after 9000 iterations the loss is 0.999234, the accuracy is 0.6628
training! after 9000 iterations the loss is 0.568378, the accuracy is 0.804688
###################################
validation! after 9100 iterations the loss is 0.995525, the accuracy is 0.6654
training! after 9100 iterations the loss is 0.422434, the accuracy is 0.867188
################################

###################################
validation! after 13000 iterations the loss is 1.06465, the accuracy is 0.6778
training! after 13000 iterations the loss is 0.162207, the accuracy is 0.976562
###################################
validation! after 13100 iterations the loss is 1.13141, the accuracy is 0.6706
training! after 13100 iterations the loss is 0.246055, the accuracy is 0.921875
###################################
validation! after 13200 iterations the loss is 1.03097, the accuracy is 0.6974
training! after 13200 iterations the loss is 0.116955, the accuracy is 0.976562
###################################
validation! after 13300 iterations the loss is 1.09468, the accuracy is 0.6832
training! after 13300 iterations the loss is 0.144529, the accuracy is 0.992188
###################################
validation! after 13400 iterations the loss is 1.11839, the accuracy is 0.6786
training! after 13400 iterations the loss is 0.143094, the accuracy is 0.96875
##########################

###################################
validation! after 17300 iterations the loss is 1.52625, the accuracy is 0.6724
training! after 17300 iterations the loss is 0.0883567, the accuracy is 0.976562
###################################
validation! after 17400 iterations the loss is 1.32603, the accuracy is 0.692
training! after 17400 iterations the loss is 0.0573429, the accuracy is 0.992188
###################################
validation! after 17500 iterations the loss is 1.39442, the accuracy is 0.695
training! after 17500 iterations the loss is 0.0145639, the accuracy is 1.0
###################################
validation! after 17600 iterations the loss is 1.41351, the accuracy is 0.6946
training! after 17600 iterations the loss is 0.0140337, the accuracy is 1.0
###################################
validation! after 17700 iterations the loss is 1.32357, the accuracy is 0.684
training! after 17700 iterations the loss is 0.0335111, the accuracy is 1.0
###################################
va